Titanic Challenge 
Great Resource: https://www.kaggle.com/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import seaborn as sns
%matplotlib inline

Let's load the data and make a copy of both training and test set to do the Data Cleaning process. 

In [44]:
train_raw = pd.read_csv('train.csv')
test_raw = pd.read_csv('test.csv')

train_data = train_raw.copy(deep=True)
test_data = test_raw.copy(deep=True)
print(train_raw.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


Now check how many missing values exist in each column

In [45]:
train_raw.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [46]:
test_raw.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Now we create and array to iterate over training and test set, so that we dont have to write 2 lines of code per operation. We will fill NaN values, create and modify features. 

In [47]:
dataset_array =[train_data, test_data]
for dataset in dataset_array:
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace=True)
    # extract title from name and create new feature named title
    dataset['Title'] = dataset['Name'].str.split(',', expand=True)[1].str.split('.', expand=True)[0]
    
print(train_data.Title.unique())


[' Mr' ' Mrs' ' Miss' ' Master' ' Don' ' Rev' ' Dr' ' Mme' ' Ms' ' Major'
 ' Lady' ' Sir' ' Mlle' ' Col' ' Capt' ' the Countess' ' Jonkheer']


In [48]:
print(test_data.Title.unique())


[' Mr' ' Mrs' ' Miss' ' Master' ' Ms' ' Col' ' Rev' ' Dr' ' Dona']


In [49]:
# get a boolean series where true = relevant title, false = irrelevant title
title_bool = (train_data['Title'].value_counts() < 10)
print(title_bool)
for dataset in dataset_array:
    dataset['Title'] = dataset['Title'].apply(lambda x: x if (x in title_bool.axes & title_bool.loc[x] != True)else 'Misc')
    
mean_per_title = train_data.groupby('Title')['Age'].mean()
print(mean_per_title)

for dataset in dataset_array:
    dataset['Age'] = dataset['Title'].apply(lambda x: mean_per_title.loc[x])

 Mr              False
 Miss            False
 Mrs             False
 Master          False
 Dr               True
 Rev              True
 Col              True
 Major            True
 Mlle             True
 Ms               True
 Capt             True
 Lady             True
 Mme              True
 the Countess     True
 Sir              True
 Jonkheer         True
 Don              True
Name: Title, dtype: bool


TypeError: unsupported operand type(s) for &: 'str' and 'bool'